### 2023: Week 49 - Regular Savings Accounts
December 06, 2023
Challenge by: Jenny Martin

Data Source Bank (DSB) are looking to introduce a Regular Savings Account for their customers. There are 2 factors they need to decide:

The interest rate
The maximum amount a customer is able to pay into the account each month
DSB are looking to compare other 12 month products currently in the market, to help decide these factors. They want to understand which account offers the customer the biggest opportunity to increase their savings. They also want to educate their customers by showing how their savings would increase each month. 



### Requirements 
Note: We assume the interest will be spread evenly across the 12 months for this challenge
 - Input the data
 - Update the Max Monthly Amount so it is a numeric field
 - Update the Provider field so it is easy for customers to see in 1 field which accounts have additional conditions
 - Ensure we have a row for each month for each account i.e. 12 rows for each account
 - Calculate the savings value in each account, each month after interest has been applied
 - Assume a customer is able to pay in the Max Monthly Deposit each month
 - Compound interest may come in useful (formula taken from Wikipedia)
 - Hint: think about each month's deposit and how many months it will collect interest before the end of the 12 months 
 - Create a field for the maximum possible savings a customer could get from each account
 - Calculate the maximum total interest a customer could get from each account
 - Rank the accounts by the maximum possible savings
 - Rank the accounts by the maximum total interest
 - Round all monetary values to 2 decimal places
 - Output the data


In [553]:
import os
import pandas as pd
import numpy as np

In [554]:
# Input the data

df = pd.read_csv('Regular Savings Accounts.csv')

In [555]:
# Clean data
df.columns=df.columns.str.lower().str.strip().str.replace(' ','_')
df.interest=df.interest.str.strip('%').astype('float64')
df.max_monthly_deposit=df.max_monthly_deposit.str.strip('£').astype('int64')
df.has_additional_conditions=df.has_additional_conditions.fillna('N')
df.provider=np.where(df.has_additional_conditions=='Y',df.provider+ ' (Conditions Apply)',df.provider)
# df.provider=df.provider + ' - '+ df.has_additional_conditions
df.drop(columns=['has_additional_conditions'],inplace=True)

In [556]:
# Create 12 rows per account

months_df=pd.DataFrame(list(range(1,13)),columns=['month'])
months_df['key']=1
df['key']=1
df=df.merge(months_df,how='outer',on='key').drop(columns=['key']).copy()

In [557]:
#  - Calculate the savings value in each account, each month after interest has been applied
#  - Assume a customer is able to pay in the Max Monthly Deposit each month
#  - Compound interest may come in useful (formula taken from Wikipedia)
#  - Hint: think about each month's deposit and how many months it will collect interest before the end of the 12 months 
#  - Create a field for the maximum possible savings a customer could get from each account
#  - Calculate the maximum total interest a customer could get from each account

def calculate_compound_interest(principal,rate,compounding_frequency,month):
  """
  principal=Principle sum
  rate=Nominal annual interest rate
  compounding_frequency=compounding frequency
  month=month of the year
  """
  rate_decimal=rate/100
  # month=13-month

  # compound_interest = principal * (1 + rate_decimal / compounding_frequency) ** (compounding_frequency * time)
  compound_interest = principal * (1 + rate_decimal / compounding_frequency) ** (month)

  return compound_interest

In [558]:
df['saving_each_month']=df.apply(lambda row: calculate_compound_interest(
  principal=row['max_monthly_deposit'],
  rate=row['interest'],
  compounding_frequency=12,
  month=row['month']
),axis=1)

df['interest_earned']=df.saving_each_month-df.max_monthly_deposit


total_savings_df = df[['provider','saving_each_month']].groupby('provider').sum().rename(columns={'saving_each_month':'total_max_savings'})
total_interest_df = df[['provider','interest_earned']].groupby('provider').sum().rename(columns={'interest_earned':'total_interest'})

df['saving_each_month']=df.groupby('provider').cumsum()[['saving_each_month']]

df=df.merge(total_savings_df,how='left',on='provider')
df=df.merge(total_interest_df,how='left',on='provider')

In [559]:
# Rank the accounts by the maximum possible savings
savings_rank=df.groupby(['provider']).sum(numeric_only=True)['total_max_savings'].rank(ascending=False).reset_index().rename(columns={'total_max_savings':'savings_rank'})

# Rank the accounts by the maximum total interest
interest_rank=df.groupby(['provider']).sum(numeric_only=True)['total_interest'].rank(ascending=False).reset_index().rename(columns={'total_interest':'interest_rank'})

df=df.merge(savings_rank,how='left',on='provider')
df=df.merge(interest_rank,how='left',on='provider')



In [560]:
# Round all monetary values to 2 decimal places
for column in df.columns:
  if df[column].dtype == 'float64':
    df[column]=df[column].round(2)

df.savings_rank=df.savings_rank.astype('int64')
df.interest_rank=df.interest_rank.astype('int64')


In [561]:
df=df[[
  'savings_rank',
  'interest_rank',
  'provider',
  'interest',
  'max_monthly_deposit',
  'month',
  'saving_each_month',
  'total_max_savings',
  'total_interest']]


In [562]:
df=df.sort_values(['savings_rank','provider']).reset_index(drop=True)

In [564]:
df.to_excel('savings accounts.xlsx',index=False)
os.startfile('savings accounts.xlsx')